In [31]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [21]:
import plotly.express as px

In [123]:
bank_data = pd.read_csv("/workspaces/dev/SVM/bank_data.txt",sep=';', quotechar='"')
bank_data.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [124]:
bank_data.shape

(41188, 21)

In [125]:
bank_data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [126]:
# Check outliers for those numerical columns with high number of unique values
px.box(bank_data["duration"])

In [127]:
# Removing rows with outliers 
Q1 = bank_data["duration"].quantile(0.25)
Q3 = bank_data["duration"].quantile(0.75)
IQR = Q3 - Q1     

filter = (bank_data["duration"] >= Q1 - 1.5 * IQR) & (bank_data["duration"] <= Q3 + 1.5 *IQR)
bank_data=bank_data.loc[filter].reset_index(drop=True)

In [128]:
bank_data.shape

(38225, 21)

DATA CLEANING AND PRE PROCESSING

In [129]:
# Removing dots inbetween words and Replacing 999 in pdays as 0
bank_data["job"] = bank_data["job"].str.replace(r'\.', '', regex=True)
bank_data["education"] = bank_data["education"].str.replace(r'\.', ' ', regex=True)
bank_data["pdays"] = bank_data["pdays"].replace(999,0)

In [130]:
# Clubbing 3 basic categories in education column into one category "middle school"
basic = ['basic 4y','basic 6y', 'basic 9y']
for i in basic :
    bank_data.loc[bank_data["education"]== i, "education"] = "middle school"

In [131]:
# Mapping boollean kind of string values to numbers
month_dict = {'mar':3, 'apr':4,'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
day_dict = {'mon':2, 'tue':3, 'wed':4, 'thu':5, 'fri':6}
dic = {'yes':1,'no':0,'unknown':-1}
bank_data["month"] = bank_data["month"].map(month_dict)
bank_data["day_of_week"] = bank_data["day_of_week"].map(day_dict)
lis = ['default', 'housing', 'loan','y']
for i in lis :
    bank_data[i] = bank_data[i].map(dic)

In [132]:
# Frequency Encoding - Categorical Data columns with high number of categories 
job_freq = bank_data["job"].value_counts().to_dict()
edu_freq = bank_data["education"].value_counts().to_dict()
# Categorical value replaced with frequency of occurence of each category
bank_data["job"] = bank_data["job"].map(job_freq)
bank_data["education"] = bank_data["education"].map(edu_freq)

In [133]:
# Encoding Categorical Data Columns using OneHot Encoder
categorical_columns = ['marital','contact', 'poutcome']
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
# Fit and transform categorical columns
encoded_categories = encoder.fit_transform(bank_data[categorical_columns])
# Convert the encoded array to a DataFrame
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(categorical_columns))

In [108]:
bank_data.drop(columns=['marital','contact', 'poutcome'],inplace=True)

In [113]:
# Combine the Dataframes
combined_df = pd.concat([encoded_df, bank_data], axis=1)

In [ ]:
combined_df

TRAINING THE MODEL

In [23]:
x = combined_df.iloc[:,:-1]
y = combined_df.iloc[:,-1]

In [26]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [27]:
# Preprocess the data by scaling features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [30]:
# Hyper Parameter Tuning
# Define the parameter grid
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],    # You can add more kernels
    'C': [0.1, 1, 10],                      # Regularization parameter
    'gamma': ['scale', 'auto']              # For RBF and polynomial kernels
}

# Initialize the SVC
svm = SVC()

# Use GridSearchCV to find the best kernel and parameters
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Best parameters and kernel
print("Best parameters:", grid_search.best_params_)

Best parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}


In [31]:
# Training the model
svc_classifier = SVC(kernel='linear', C=0.1, gamma='scale', random_state=42)
svc_classifier.fit(x_train, y_train)

SVC(C=0.1, kernel='linear', random_state=42)

In [32]:
# Make predictions on the test set
y_pred = svc_classifier.predict(x_test)

In [33]:
# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [34]:
# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10935
           1       1.00      1.00      1.00      1422

    accuracy                           1.00     12357
   macro avg       1.00      1.00      1.00     12357
weighted avg       1.00      1.00      1.00     12357

